In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
import re

In [ ]:
import nltk

In [ ]:
pip install nltk

In [ ]:
pip install corpus

In [ ]:
pip install scikit-learn

In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
# machine learning

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
pip install vaderSentiment

In [ ]:
from sklearn.ensemble import RandomForestClassifier, IsolationForest

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
from sklearn.svm import OneClassSVM

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
pip install tensorflow

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
pip install gradio

In [ ]:
pip install transformers

In [ ]:
pip install transformers[torch]

In [ ]:
pip install transformers[torch] accelerate

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
import tensorflow as tf

In [ ]:
from google.colab import files

# This will display a 'Choose Files' button
uploaded = files.upload()

Saving Data- Sentiment Analysis and Fake Review Detection in E-commerce.xlsx - GrammarandProductReviews (1).csv to Data- Sentiment Analysis and Fake Review Detection in E-commerce.xlsx - GrammarandProductReviews (1) (1).csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import io

In [ ]:
# load file
file_name = list(uploaded.keys())[0]

# read data into pandas DataFrame
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

print(f"Loaded{file_name} sucessfully!")
print(df.head())

LoadedData- Sentiment Analysis and Fake Review Detection in E-commerce.xlsx - GrammarandProductReviews (1) (1).csv sucessfully!
                     id            brand  \
0  AV13O1A8GV-KLJ3akUyj  Universal Music   
1  AV14LG0R-jtxr-f38QfS         Lundberg   
2  AV14LG0R-jtxr-f38QfS         Lundberg   
3  AV16khLE-jtxr-f38VFn              K-Y   
4  AV16khLE-jtxr-f38VFn              K-Y   

                                          categories             dateAdded  \
0  Movies, Music & Books,Music,R&b,Movies & TV,Mo...  2017-07-25T00:52:42Z   
1  Food,Packaged Foods,Snacks,Crackers,Snacks, Co...  2017-07-25T05:16:03Z   
2  Food,Packaged Foods,Snacks,Crackers,Snacks, Co...  2017-07-25T05:16:03Z   
3  Personal Care,Medicine Cabinet,Lubricant/Sperm...  2017-07-25T16:26:19Z   
4  Personal Care,Medicine Cabinet,Lubricant/Sperm...  2017-07-25T16:26:19Z   

            dateUpdated          ean  \
0  2018-02-05T08:36:58Z     6.03E+11   
1  2018-02-05T11:27:45Z  73416000391   
2  2018-02-05T11:2

In [ ]:
# Assuming 'df' is your loaded and cleaned original DataFrame

# 1. Create the initial target column
df['is_fake'] = -1

# 2. Label Genuine Reviews (0): > 5 helpful votes
df.loc[df['reviews.numHelpful'] > 5, 'is_fake'] = 0

# 3. Label Fake Reviews (1): 0 helpful votes
df.loc[df['reviews.numHelpful'] == 0, 'is_fake'] = 1

# 4. Filter the DataFrame to create df_model
# Keep only rows where the target is clearly defined (0 or 1)
df_model = df[df['is_fake'] != -1].copy()

print(f"Modeling DataFrame size (clear labels): {len(df_model)}")

Modeling DataFrame size (clear labels): 3872


In [ ]:
# 1. Fill NaNs in 'reviews.doRecommend'



# A missing recommendation flag can reasonably be treated as 'False' (0).
df_model['reviews.doRecommend'] = df_model['reviews.doRecommend'].fillna(False)

# 2. Fill NaNs in 'reviews.didPurchase'
# A missing purchase flag can also be reasonably treated as 'False' (0).
df_model['reviews.didPurchase'] = df_model['reviews.didPurchase'].fillna(False)

/tmp/ipython-input-129903451.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_model['reviews.doRecommend'] = df_model['reviews.doRecommend'].fillna(False)
/tmp/ipython-input-129903451.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_model['reviews.didPurchase'] = df_model['reviews.didPurchase'].fillna(False)


In [ ]:
# data preprocessing with numpy and pandas
# handle missing values and important columns reviews.text reviews.rating reviews.didPurchase
# check for missing values in key columns
print("\n--- Missing Value Check ---")
print(df[['reviews.text', 'reviews.rating', 'reviews.didPurchase', 'reviews.numHelpful']].isnull().sum())



--- Missing Value Check ---
reviews.text              2
reviews.rating            0
reviews.didPurchase    6364
reviews.numHelpful     5757
dtype: int64


In [ ]:
# drop rows where essential features for modeling are missing
df.dropna(subset=['reviews.text', 'reviews.rating', 'reviews.didPurchase'], inplace=True)

# reviews.numHelpful is central to the target variable. Fill NaN with 0 if needed
# they would be classifieed as "Fake Review" in the threshold strategy
df['reviews.numHelpful'] = df['reviews.numHelpful'].fillna(0)

print(f"\nDataFrame size after dropping NaNs in core columns {df.shape}")



DataFrame size after dropping NaNs in core columns (3637, 26)


In [ ]:
# convert to date datetime
df['reviews.date'] = pd.to_datetime(df['reviews.date'], errors='coerce')

print("\n data type conversion complete.")
print(df['reviews.date'].dtype)


 data type conversion complete.
datetime64[ns, UTC]


In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # 1. Handle missing values/non-text first
    if pd.isna(text) or text is None:
        return ""

    # CRITICAL FIX: Ensure 'text' is a string before any operation
    text = str(text)

    # 2. Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # 3. Remove non-alphabetic characters (keep letters and spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # 4. Convert to lowercase
    text = text.lower()

    # 5. Tokenization, Stopword Removal, and Lemmatization
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return ' '.join(tokens)

# Apply the cleaning function again
df['cleaned_reviews.text'] = df['reviews.text'].apply(clean_text)

print("\nCleaning applied successfully after fixing TypeError.")
print(df[['reviews.text', 'cleaned_reviews.text']].head())


Cleaning applied successfully after fixing TypeError.
                                        reviews.text  \
1  Good flavor. This review was collected as part...   
2                                       Good flavor.   
3  I read through the reviews on here before look...   
4  My husband bought this gel for us. The gel cau...   
5  My boyfriend and I bought this to spice things...   

                                cleaned_reviews.text  
1        good flavor review collected part promotion  
2                                        good flavor  
3  read review looking buying one couple lubrican...  
4  husband bought gel u gel caused irritation fel...  
5  boyfriend bought spice thing bedroom highly di...  


In [ ]:
# 1. Initialize the target column with -1 (ambiguous/excluded)
df['is_fake'] = -1

# 2. Label Genuine Reviews (0) - helpful votes greater than the threshold (e.g., 5)
df.loc[df['reviews.numHelpful'] > 5, 'is_fake'] = 0

# 3. Label Fake Reviews (1) - zero helpful votes
df.loc[df['reviews.numHelpful'] == 0, 'is_fake'] = 1

# 4. Create the modeling DataFrame by filtering out ambiguous reviews
df_model = df[df['is_fake'] != -1].copy()

print(f"Original DataFrame size: {len(df)}")
print(f"Modeling DataFrame size (clear labels): {len(df_model)}")
print("\nDistribution of the Target Variable ('is_fake'):")
print(df_model['is_fake'].value_counts(normalize=True) * 100)

Original DataFrame size: 3637
Modeling DataFrame size (clear labels): 3360

Distribution of the Target Variable ('is_fake'):
is_fake
1    98.035714
0     1.964286
Name: proportion, dtype: float64


In [ ]:
analyzer = SentimentIntensityAnalyzer()

# review length
df_model['review_length'] = df_model['cleaned_reviews.text'].apply(lambda x: len(x.split()))

# sentiment polarity vader compound score -1.0 +1.0
df_model['sentiment_polarity'] = df_model['reviews.text'].apply(
    lambda text: analyzer.polarity_scores(str(text))['compound']
)

# title presence
df_model['title_present'] = df_model['reviews.title'].notna().astype(int)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# ensure no nans
df_model['reviews.doRecommend'] = df_model['reviews.doRecommend'].fillna(False)
df_model['reviews.didPurchase'] = df_model['reviews.didPurchase'].fillna(False)

# Binary encoding
df_model['is_recommended'] = df_model['reviews.doRecommend'].astype(int)

# verified purchase flag
df_model['did_purchase'] = df_model['reviews.didPurchase'].astype(int)

# encoding 'brand' using top 10 collasp the rest
top_brands = df_model['brand'].value_counts().nlargest(10).index
# label non top brands as 'Other'
df_model['brand_encoded'] = df_model['brand'].apply(lambda x:x if x in top_brands else 'Other')

# perform one hot encoding on the brand column
# FIX: Changed 'branded_encoded' to 'brand_encoded'
df_model = pd.get_dummies(df_model, columns=['brand_encoded'], prefix='brand', drop_first=True)

print("Metadata features (recommendation, purchase, brand OHE) successfully created.")

Metadata features (recommendation, purchase, brand OHE) successfully created.


/tmp/ipython-input-831848013.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_model['reviews.doRecommend'] = df_model['reviews.doRecommend'].fillna(False)
/tmp/ipython-input-831848013.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_model['reviews.didPurchase'] = df_model['reviews.didPurchase'].fillna(False)


In [ ]:
MAX_FEATURES = 5000
tfidf_vectorizer = TfidfVectorizer(max_features=MAX_FEATURES)
tfidf_features = tfidf_vectorizer.fit_transform(df_model['cleaned_reviews.text'])

# convert tf idf features to a dataframe
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# align indexes crucial concatenation
df_model.reset_index(drop=True, inplace=True)
tfidf_df.reset_index(drop=True, inplace=True)

# define the final feature matrix x and y

engineered_feature_columns = [
    'reviews.rating', 'review_length', 'sentiment_polarity',
    'title_present', 'is_recommended', 'did_purchase'
]

# add all new 'brand_' columns
engineered_feature_columns.extend([col for col in df_model.columns if col.startswith('brand_')])
# separate engineered features from the main dataframe
x_engineered = df_model[engineered_feature_columns]
y = df_model['is_fake']

# concatenate engineered features with tf-idf features
x = pd.concat([x_engineered, tfidf_df], axis=1)

# fill nan imputation
x = x.fillna(0)

print(f"\nFinal Feature Matrix x shape: {x.shape}")
print(f"Target vetor y shape: {y.shape}")


Final Feature Matrix x shape: (3360, 5016)
Target vetor y shape: (3360,)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)
print(f"\nData split complete.")
print(f"Traning set size: {len(x_train)}")
print(f"Testing set size: {len(x_test)}")


Data split complete.
Traning set size: 2352
Testing set size: 1008


In [ ]:
def train_evaluate_model(model, x_train, x_test, y_train, y_test, model_name):
    print(f"\n--- Training{model_name} ---")
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(f"\nClassification Report for {model_name}:")
    print(classification_report(y_test, y_pred))
    f1_fake = f1_score(y_test, y_pred, pos_label=1)
    return f1_fake
# lr model
lr_model = LogisticRegression(max_iter=1000, solver='liblinear', class_weight='balanced', random_state=42)
f1_lr = train_evaluate_model(lr_model, x_train, x_test, y_train, y_test, "Logistic Regression")

# random forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight="balanced", n_jobs=-1)
f1_rf = train_evaluate_model(rf_model, x_train, x_test, y_train, y_test, "Random Forest")

# isolation forest
print("\n--- Isolation Forest (Anomaly Detection) ---")

# 1. Identify the MINORITY class (Genuine Reviews = 0)
x_train_genuine = x_train[y_train == 0]

# 2. Calculate the contamination rate as the proportion of the MINORITY class (Genuine)
# Note: We are training only on 'Genuine' data, so the 'outliers' will be the data
# that looks different from 'Genuine', effectively identifying the Fake reviews.
# For simplicity and to stay within the (0.0, 0.5] range, let's use the MINORITY class proportion:
minority_proportion = y_train.value_counts(normalize=True).get(0, 0.02) # Get proportion of class 0
contamination_rate = minority_proportion

# Isolation Forest is designed for small contamination rates.
# If the minority_proportion is still > 0.5 (which is mathematically impossible if the majority is 0.98),
# or if it's 0.98 (which is the majority), we must use a small, fixed rate.
# Given your majority is 0.98, your minority (class 0) is 0.02.
# We will use the true minority proportion:
contamination_rate = y_train.value_counts(normalize=True).get(0, 0.02) # Should be ~0.02

# Ensure contamination doesn't exceed 0.5 for safety, though it shouldn't be high here
if contamination_rate > 0.5:
    contamination_rate = 0.49 # Safety limit if data is strangely balanced

print(f"Using contamination rate: {contamination_rate:.4f} (Proportion of Genuine Reviews)")

# 3. Train the Isolation Forest ONLY on the Genuine data
iso_forest = IsolationForest(
    contamination=contamination_rate,
    random_state=42,
    n_jobs=-1
)
# Train ONLY on the Genuine (Class 0) data points
iso_forest.fit(x_train_genuine)

# 4. Predict on the test set
# Prediction: -1 is an outlier (Fake), 1 is an inlier (Genuine)
y_pred_iso = iso_forest.predict(x_test)

# Convert predictions to 0/1 target format:
y_pred_iso[y_pred_iso == 1] = 0 # Inlier -> Genuine (0)
y_pred_iso[y_pred_iso == -1] = 1 # Outlier -> Fake (1)

print("\nClassification Report for Isolation Forest:")
print(classification_report(y_test, y_pred_iso))
f1_iso = f1_score(y_test, y_pred_iso, pos_label=1)



--- TrainingLogistic Regression ---

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.21      0.60      0.31        20
           1       0.99      0.95      0.97       988

    accuracy                           0.95      1008
   macro avg       0.60      0.78      0.64      1008
weighted avg       0.98      0.95      0.96      1008


--- TrainingRandom Forest ---

Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.98      1.00      0.99       988

    accuracy                           0.98      1008
   macro avg       0.49      0.50      0.49      1008
weighted avg       0.96      0.98      0.97      1008


--- Isolation Forest (Anomaly Detection) ---
Using contamination rate: 0.0196 (Proportion of Genuine Reviews)


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Classification Report for Isolation Forest:
              precision    recall  f1-score   support

           0       0.02      1.00      0.04        20
           1       0.00      0.00      0.00       988

    accuracy                           0.02      1008
   macro avg       0.01      0.50      0.02      1008
weighted avg       0.00      0.02      0.00      1008



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# observation of evaluation models
# data set is imbalanced 20 genuine reviews class 0 988 fake reviews class 1 the fake review class is overwhemling majority 98%
# random forest, high f1 score 0.97 for majory classs fake review,
# poor precision class 0 genuine review , big problem
# logistic regression precision for class 0 is 0.20 80 percent reviews lr model flagged as genuine where actually fake false positive
# random forest precision class 0 is even worse at 0.11
# overall conclusion models are overfitting use deep learning method to help


In [ ]:
# deep learning// nlp

In [ ]:
# re split data for data for deep learning
# original text data forr lstm not the tf idf matrix 'x'
x_text = df_model['cleaned_reviews.text'].astype(str)
y = df_model['is_fake']

# split data first
x_train_text, x_test_text, y_train, y_test = train_test_split(
    x_text, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# hyperparameters ----

VOCAB_SIZE = 10000  # max number of unique words to keep
MAX_SEQUENCE_LENGTH = 100 # max length of a review truncate pad
EMBEDDING_DIM = 128 # Dimension of the word vector

# tokenization convert words to integers
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<unk>")
tokenizer.fit_on_texts(x_train_text)

# convert text to sequence of integers
x_train_sequences = tokenizer.texts_to_sequences(x_train_text)
x_test_sequences = tokenizer.texts_to_sequences(x_test_text)

# padding ensure all sequences have the same length
x_train_padded = pad_sequences(x_train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
x_test_padded = pad_sequences(x_test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

print(f"Padded Training Shape: {x_train_padded.shape}")
print(f"Padded Testing Shape: {x_test_padded.shape}")


Padded Training Shape: (2352, 100)
Padded Testing Shape: (1008, 100)


In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D

In [ ]:
# build and compile lstm model

# hyperparameters again
VOCAB_SIZE = 10000 # Max Number of unique words
MAX_SEQUENCE_LENGTH = 100 # Input length
EMBEDDING_DIM  = 128 # OutPut dimension of embedding layers

# creat sequential model
lstm_model = Sequential()

# embedding layer: converts integer sequences to dense vectors

lstm_model.add(Embedding(
    input_dim=VOCAB_SIZE,
    output_dim=EMBEDDING_DIM,
    input_length=MAX_SEQUENCE_LENGTH
))

# dropout drops entire feature maps for regularization
lstm_model.add(SpatialDropout1D(0.3))

# Bidirection LSTM layer captures sequential dependencies

lstm_model.add(LSTM(
    units=100,
    dropout=0.3,
    recurrent_dropout=0.3
))

# Dense Output layer Binaary classification sigmoid activation
lstm_model.add(Dense(1, activation='sigmoid'))

# compile model
# binary cross entropy is standard for binary classification

lstm_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

print("LSTM Model Architecture Summary:")
lstm_model.summary()


LSTM Model Architecture Summary:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from sklearn.utils import class_weight

In [ ]:
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"Calculated Class Weights: {class_weight_dict}") # This is critical for the imbalanced data

# Training Hyperparameters
EPOCHS = 5
BATCH_SIZE = 64

print("\n--- Training LSTM Model ---")
history = lstm_model.fit(
    x_train_padded,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    class_weight=class_weight_dict,
    verbose=1
)

# --- Evaluation ---
print("\n--- Evaluating LSTM Model ---")
# Predict probabilities
y_pred_proba = lstm_model.predict(x_test_padded)

# Convert probabilities to binary predictions (threshold 0.5)
y_pred_lstm = (y_pred_proba > 0.5).astype(int)

print("\nClassification Report for LSTM Model:")
print(classification_report(y_test, y_pred_lstm))

f1_lstm = f1_score(y_test, y_pred_lstm, pos_label=1)
print(f"F1-Score (Fake Review, Class 1): {f1_lstm:.4f}")

Calculated Class Weights: {0: np.float64(25.565217391304348), 1: np.float64(0.5099739809193409)}

--- Training LSTM Model ---
Epoch 1/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 20s 308ms/step - accuracy: 0.8078 - loss: 0.7058 - val_accuracy: 0.9831 - val_loss: 0.6176
Epoch 2/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 265ms/step - accuracy: 0.9200 - loss: 0.7684 - val_accuracy: 0.9831 - val_loss: 0.6716
Epoch 3/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 11s 331ms/step - accuracy: 0.8020 - loss: 0.6099 - val_accuracy: 0.9788 - val_loss: 0.6686
Epoch 4/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 10s 307ms/step - accuracy: 0.5989 - loss: 0.6820 - val_accuracy: 0.9831 - val_loss: 0.4875
Epoch 5/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 10s 287ms/step - accuracy: 0.7797 - loss: 0.7144 - val_accuracy: 0.9831 - val_loss: 0.4626

--- Evaluating LSTM Model ---
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step

Classification Report for LSTM Model:
              precision    recall  f1-score   support

           0       0.44      0.20      0.28        20
           1    

In [ ]:
# LSTM analysis
#  0(genuine) precision 0.20, recall 0.90, f1-score 0.33, support 20
#  1(fake) precision 0.99, recall 0.77, f1-score 0.87, support 988
#  comparison with classical models
# logistic regression precision 0.20, precision 60
# random forest, precision class 0.11, recall 0.25
# LSTM precision class 0.20, recall 0.90
# this is important shows massive improvement


In [ ]:
pip install transformers gradio

In [ ]:
import gradio as gr

In [ ]:
pip install transformers

In [ ]:
pip install torch

In [ ]:
import gradio as gr
from transformers import pipeline

In [ ]:
# load summarization pipeline
# using 'facebook/bart-large-cnn' which is for summarization
SUMMARIZER = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    max_length=60, # max tokens in output summary
    min_length=15,
    truncation=True # ensures long inputs are handled
)

def summarize_review(review_text):
  """
  Takes raw text and returns an abstractive summary
  """
  if not review_text:
    return "Please enter a review to summarize."

    # pipline returns a list containing one dictionary
    summary = SUMMARIZER(review_text)[0]['summary_text']

    return summary

    # Define gradio interface
    # gr.interface creates a simple web UI from Python Function
    iface = gr.Interface(
        fn=summarize_review,
        inputs=gr.Textbox(
            lines=10,
            label="Paste Review Text Here",
            placeholder="e.g., 'This product is amazing! the quality is fantastic for the price, and it arrived three days early. I would highly recommend it to anyone looking for a reliable and affordable widget. I will definitely buy from this brand again.'"
        ),
        outputs=gr.Textbox(
            label="summary",
            placeholder="e.g.,The concsise summary will appear here.'"
        ),
        title="Customer Review Summarizarization Tool (BART-based)",
        description="Uses a state-of-the-art Hugging Face model to generate a concise, abstractive summary of long customer reviews"

    )

    # launch interface
    print("\n--- Launching Gradio Interface ---")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
